# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
path = '../data/Orders.csv'

In [3]:
Orders = pd.read_csv(path)
Orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# Sub Problem 1:
Amount_spent = Orders.groupby(by=["CustomerID"]).sum()
Amount_spent.sort_values(by="amount_spent",ascending = False).head(5)

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
14646,544561120,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02
18102,138022684,243297801,866723,3746,1261,5587,64124,1940.92,259657.30
17450,92683919,188845149,677704,2292,842,4140,69993,1143.32,194550.79
16446,929130,1688629,6033,22,11,27,80997,4.98,168472.50
14911,1737367680,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06


In [5]:
# Sub Problem 2 + 3:
# boundaries = range of q (bins are for cut)
# define labels
# VIP: above 95%
# Preferred: 75-95%
# Rest: casual
bin_df = pd.qcut(Orders['amount_spent'], q = [0,0.75,0.95,1], labels=["casual", "preferred", "VIP"]).sort_values()
bin_df

0         casual
251225    casual
251224    casual
251223    casual
251222    casual
           ...  
164341       VIP
121878       VIP
286383       VIP
137762       VIP
23974        VIP
Name: amount_spent, Length: 397924, dtype: category
Categories (3, object): ['casual' < 'preferred' < 'VIP']

In [6]:
bin_df.value_counts()

casual       308302
preferred     70007
VIP           19615
Name: amount_spent, dtype: int64

In [7]:
Orders["Label"]= bin_df
Orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,casual
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,preferred


In [8]:
Orders_1 = Orders.copy()

In [9]:
# Cut solution:
# variable_name = pd.cut(Orders["amount_spent"], 
                             # bins = [Orders["amount_spent"].quantile(0.),Orders["amount_spent"].quantile(.75),Orders["amount_spent"].quantile(.95),Orders["amount_spent"].quantile(1.)],
                            # labels=["Casual", "Preferred","VIP"])

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [10]:
# your code here
# groupy country, .count column of VIPS
# df.groupby(column to group by).count()
# df[df[vip column]== VIP].groupby(country).count()

In [16]:
Orders_1.groupby(["Label", "Country"]).size()

Label      Country             
casual     Australia                  342
           Austria                    280
           Bahrain                      6
           Belgium                   1450
           Brazil                      17
           Canada                     118
           Channel Islands            449
           Cyprus                     420
           Czech Republic               5
           Denmark                    163
           EIRE                      4576
           European Community          37
           Finland                    408
           France                    5606
           Germany                   6149
           Greece                      97
           Iceland                    115
           Israel                     130
           Italy                      492
           Japan                       80
           Lebanon                     18
           Lithuania                    8
           Malta                       82
  

In [12]:
# The UK has the most  VIP customers

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [14]:
pd.set_option('display.max_rows', 500)

In [15]:
Orders_1.groupby(["Label", "Country"]).size()

Label      Country             
casual     Australia                  342
           Austria                    280
           Bahrain                      6
           Belgium                   1450
           Brazil                      17
           Canada                     118
           Channel Islands            449
           Cyprus                     420
           Czech Republic               5
           Denmark                    163
           EIRE                      4576
           European Community          37
           Finland                    408
           France                    5606
           Germany                   6149
           Greece                      97
           Iceland                    115
           Israel                     130
           Italy                      492
           Japan                       80
           Lebanon                     18
           Lithuania                    8
           Malta                       82
  

In [ ]:
# The UK has the most  VIP+ preferred customers